# Analisis de comentarios de películas

## 1. Carga de datos

## 2. Preparación de los datos

Para preparar tus datos para utilizar el algoritmo de Naive Bayes en el análisis de sentimientos, necesitas realizar algunas tareas de preprocesamiento de los datos. Los pasos típicos de preprocesamiento incluyen:

- Limpiar el texto: esto implica eliminar cualquier información no relevante para el análisis de sentimientos, como signos de puntuación, números, caracteres especiales y palabras vacías (como "a", "el", "y", etc.). Puedes hacer uso de librerías de procesamiento de texto como NLTK o SpaCy para llevar a cabo esta tarea.
- Tokenizar el texto: esto implica dividir el texto en unidades más pequeñas, como palabras individuales. Puedes hacer uso de las mismas librerías que en el paso anterior.
- Crear una matriz de términos: una vez que tienes tus textos limpios y tokenizados, necesitas crear una matriz que represente la frecuencia de cada término en cada texto. Puedes hacer uso de la clase CountVectorizer de Scikit-Learn para crear esta matriz.
- Dividir los datos en conjuntos de entrenamiento y prueba: debes dividir tus datos en conjuntos de entrenamiento y prueba para poder evaluar el rendimiento del modelo. Puedes hacer uso de la función train_test_split de Scikit-Learn para llevar a cabo esta tarea.
- Aplicar una transformación Tfidf: Puedes aplicar una transformación Tf-idf a la matriz de términos que creaste en el paso 3, esto ayuda a reducir el peso de palabras muy comunes que no aportan información valiosa al modelo. Scikit-Learn tiene una clase TfidfTransformer que puedes utilizar para aplicar esta transformación.
- Una vez que hayas completado estos pasos, estarás listo para entrenar y evaluar tu modelo de Naive Bayes. Puedes seguir los pasos que mencioné anteriormente para utilizar la implementación de Scikit-Learn del algoritmo de Naive Bayes para clasificación de texto.

Claro, para realizar el primer paso de la preparación de los datos, que es la limpieza del texto, puedes seguir los siguientes pasos:

- Eliminar signos de puntuación: utiliza expresiones regulares para eliminar signos de puntuación como comas, puntos, comillas, etc. Estos signos de puntuación no aportan información valiosa para el análisis de sentimientos.
- Eliminar caracteres especiales: elimina cualquier otro carácter especial que no sea un signo de puntuación, como @, #, $, etc.
- Convertir todo el texto a minúsculas: esto ayuda a evitar problemas de inconsistencia en el texto, ya que "Hola" y "hola" se considerarán diferentes palabras si no se convierten a minúsculas.
- Eliminar números: los números no aportan información relevante para el análisis de sentimientos, por lo que es una buena práctica eliminarlos.
- Eliminar palabras vacías: las palabras vacías, como "a", "el", "y", "o", etc., son muy comunes en el lenguaje natural pero no aportan información valiosa para el análisis de sentimientos. Puedes hacer uso de librerías de procesamiento de texto como NLTK o SpaCy para eliminar estas palabras.
- Aplicar stemming: stemming es el proceso de reducir las palabras a su forma base o raíz. Esto ayuda a reducir la cantidad de palabras diferentes que se usan en un texto y simplifica el análisis. NLTK tiene una clase SnowballStemmer que puedes utilizar para llevar a cabo esta tarea.
- Una vez que hayas completado estos pasos, tendrás tus textos limpios y listos para el siguiente paso en la preparación de los datos.

Para eliminar palabras vacías, puedes hacer uso de las librerías NLTK o SpaCy en Python. Aquí te presento un ejemplo de cómo hacerlo con NLTK:

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

text = "Este es un ejemplo de texto con palabras vacías que deben ser eliminadas"
words = text.split()
filtered_words = [word for word in words if word.lower() not in stop_words]
filtered_text = ' '.join(filtered_words)

print(filtered_text)


Para tokenizar el texto utilizando las librerías NLTK o SpaCy en Python, puedes seguir los siguientes ejemplos:

Para tokenizar con NLTK:

In [ ]:
import nltk

text = "Este es un ejemplo de texto que vamos a tokenizar."
tokens = nltk.word_tokenize(text)

print(tokens)


## 3. Implementación algoritmos

## 3.1 Algoritmo Nathalia

## 3.2 Algoritmo Sergio

## 3.3 Algoritmo Calixto

## 4. Análisis de resultados

## 5. Conclusiones